In [12]:
import urllib.request as urllib2
from bs4 import BeautifulSoup
import pandas as pd
import re
from unidecode import unidecode
import os

In [5]:
# get the link to the metrolyrics page of your artist

page_link = 'http://www.metrolyrics.com/eminem-lyrics.html'
page = urllib2.urlopen(page_link)
soup = BeautifulSoup(page, 'html.parser')
songs = soup.find_all('a', attrs={'class': 'title hasvidtable'})

# extract all the song titles you can find
link_lyric_list = []
for song in songs:
    link_lyric = re.findall(r"(http.*html)" ,(str(song)))[0]
    link_lyric_list.append(link_lyric)
    
df_lyrics = pd.DataFrame(link_lyric_list, columns=['link'])

In [6]:
# itterate through df and get all the text from the link

for index, row in df_lyrics.iterrows():
    page = urllib2.urlopen(row['link'])
    soup = BeautifulSoup(page, 'html.parser')
    verses = soup.find_all('p', attrs={'class': 'verse'})
    title_element = soup.find_all('title')

    title = re.findall(r"(?<=<title>).*?(?=\sLyrics)" , str(title_element))[0]

    
    lyrics = ''
    for verse in verses:
        text = verse.text.strip()
        text = re.sub(r"\[.*\]\n", "", unidecode(text))
        if lyrics == '':
            lyrics = lyrics + text.replace('\n', ' ')
        else:
            lyrics = lyrics + ' ' + text.replace('\n', ' ')

    df_lyrics.at[index, 'lyrics'] = lyrics
    df_lyrics.at[index, 'title'] = title

    print('writing: ', title )

writing:  Rap God
writing:  Eminem - Lose Yourself
writing:  Eminem - Lose Yourself
writing:  Eminem - Love The Way You Lie
writing:  Eminem - The Real Slim Shady
writing:  Eminem - I'm Sorry Mama (full Song)
writing:  Eminem - Without Me
writing:  Eminem - Mockingbird
writing:  Eminem - No Love
writing:  Eminem - Not Afraid
writing:  Eminem - Stan
writing:  Eminem - Till I Collapse
writing:  Eminem - Beautiful
writing:  Eminem - I Need A Doctor
writing:  Eminem - Lighters
writing:  Eminem - Superman
writing:  Eminem - The Monster
writing:  Eminem - My Mom
writing:  Eminem - When I'm Gone
writing:  Eminem - Shake That
writing:  Eminem - The Way I Am
writing:  Eminem - Ass Like That
writing:  Eminem - My Name Is
writing:  Eminem - Cleaning Out My Closet
writing:  Eminem - Just Lose It
writing:  Eminem - Space Bound
writing:  Eminem - Fack
writing:  Eminem - Airplanes Part 2
writing:  Eminem - River
writing:  Eminem - 25 To Life
writing:  Eminem - Legacy
writing:  Eminem - 8 Mile Freesty

In [7]:
print(df_lyrics.head())

                                                link  \
0  http://www.metrolyrics.com/rap-god-lyrics-emin...   
1  http://www.metrolyrics.com/lose-yourself-lyric...   
2  http://www.metrolyrics.com/lose-yourself-dirty...   
3  http://www.metrolyrics.com/love-the-way-you-li...   
4  http://www.metrolyrics.com/the-real-slim-shady...   

                                              lyrics  \
0  Look, I was gonna go easy on you not to hurt y...   
1  Look If you had one shot, or one opportunity T...   
2  Look If you had one shot, or one opportunity T...   
3  Just gonna stand there and watch me burn But t...   
4  Eminem] May I have your attention please? May ...   

                           title  
0                        Rap God  
1         Eminem - Lose Yourself  
2         Eminem - Lose Yourself  
3  Eminem - Love The Way You Lie  
4   Eminem - The Real Slim Shady  


In [8]:
# drop columns with songs not only by Eminem
mask = df_lyrics['title'].str.contains('Eminem', regex=False)
df_lyrics = df_lyrics[mask]

# drop second row because we only need the explicit version of Lose Yourself. Parental Advisory.
df_lyrics.drop(df_lyrics.index[1], inplace=True)
print(df_lyrics.head())

                                                link  \
1  http://www.metrolyrics.com/lose-yourself-lyric...   
3  http://www.metrolyrics.com/love-the-way-you-li...   
4  http://www.metrolyrics.com/the-real-slim-shady...   
5  http://www.metrolyrics.com/im-sorry-mama-full-...   
6  http://www.metrolyrics.com/without-me-lyrics-e...   

                                              lyrics  \
1  Look If you had one shot, or one opportunity T...   
3  Just gonna stand there and watch me burn But t...   
4  Eminem] May I have your attention please? May ...   
5  Where's my snare? I have no snare on my headph...   
6  Obie Trice, real name no gimmicks Two trailer ...   

                                 title  
1               Eminem - Lose Yourself  
3        Eminem - Love The Way You Lie  
4         Eminem - The Real Slim Shady  
5  Eminem - I'm Sorry Mama (full Song)  
6                  Eminem - Without Me  


In [14]:
# check if folder exists
directory = "files"
if not os.path.exists(directory):
    os.makedirs(directory)
    

filename = directory + "/eminem_lyrics.csv"

print('writing to .csv')
df_lyrics.to_csv(filename, sep=';', encoding='utf-8')

writing to .csv
